In [1]:
from lp_coco_utils.lp_getDataset import getDatasetProcessed
from lp_training.lp_trainer import train
from lp_training.lp_loss import computeLoss
from lp_model.lp_litepose import LitePose
import lp_config.lp_common_config as cc
import torch
from lp_inference.lp_inference import inference
from lp_utils.lp_image_processing import drawHeatmap, drawKeypoints, normalizeImage
from lp_testing.lp_evaluation import computeOKS

This file has to be seen only as an entry that calls wrapper functions, the implentation of those functions can be found in the subdirectories of the repository.   
Every hyperparameter can be edited in `src/lp_config`.  
`lp_common_config.py` holds the general configurations about the dataset loading, training and test. On the other hand `lp_model_config.py` contrains the parameters that encode the model structure. The current model configs are taken from the Neural Architecture Search performed by the paper authors. I took the small size network due to the computational power available, however better results can be achieved simply by scaling the network size (Good parameters combinations are provided by the paper authors).

Code taken by the [official paper repository](https://github.com/mit-han-lab/litepose):
- classes `CocoDataset` and `CocoKeypoints` are partially taken, I added fiftyone support that makes the dataset setup easier and I removed unnecessary code.
- I took the code inside `lp_generators.py` and `lp_transforms.py` as well, since they were a `CocoKeypoints` dependencies

# Training

The dataset is downloaded by using fiftyone APIs and keypoint heatmaps are created for each sample. 

In [2]:
train(cc.config["batch_size"])

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


KeyboardInterrupt: 

# Inference
Unfotunately OpenCV method `imshow()` has a well known bug with python notebooks.

In [2]:
import cv2

model = LitePose().to(cc.config["device"])
model.load_state_dict(torch.load("lp_trained_models/crowdtest"))

ds = getDatasetProcessed("validation")

data_loader = torch.utils.data.DataLoader(
    ds,
    batch_size=8
)

row = next(iter(data_loader))
images = row[0].to(cc.config["device"])
gthm = row[1]
output, keypoints = inference(model, images)

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!


In [4]:
jointsHeatmap = output[1][2][:cc.config["num_joints"]]

img, finalHm, superimposed = drawHeatmap(images[2], jointsHeatmap)
img, gtfinalHm, gtsuperimposed = drawHeatmap(images[2], gthm[1][2])
cv2.imshow("Image", img)
cv2.imshow("Final heatmap", finalHm)
cv2.imshow("Superimposed", superimposed)

cv2.imshow("Ground Truth heatmap", gtfinalHm)
cv2.imshow("Ground Truth Superimposed", gtsuperimposed)
cv2.waitKey()

113

In [5]:
import lp_utils.lp_image_processing as ip
import numpy as np

img = images[2]

tj = output[1][2]

for t in range(14,28):
    tagJoints = tj[t]
    scaled = ip.scaleImage(tagJoints.unsqueeze(0), img.shape[1]).cpu().numpy()
    scaled = scaled[0]
    scaled = ip.normalizeImage(scaled)
    finalHm = cv2.applyColorMap(np.uint8(scaled), cv2.COLORMAP_JET)
    cv2.imshow(str(t), finalHm)
    cv2.waitKey()
cv2.destroyAllWindows()


#img = img.cpu().numpy().transpose(1, 2, 0)
#img = normalizeImage(img)
#img = np.uint8(img)
#heatmaps = ip.scaleImage(tagJoints, img.shape[1]).cpu().numpy()

#finalHm = ip.mergeMultipleHeatmaps(heatmaps)
#finalHm = normalizeImage(finalHm)
#finalHm = cv2.applyColorMap(np.uint8(finalHm), cv2.COLORMAP_JET)

#cv2.imshow("Final heatmap", finalHm)


: 

In [ ]:
tagJoints = output[1][2][cc.config["num_joints"]:]

print(tagJoints[0].max())

img, finalHm, superimposed = drawHeatmap(images[2], tagJoints)
cv2.imshow("Image", img)
cv2.imshow("Final heatmap", finalHm)
cv2.imshow("Superimposed", superimposed)

cv2.waitKey()
cv2.destroyAllWindows()

In [3]:
img = drawKeypoints(images[2], keypoints[2])
cv2.imshow("Image Keypoints", img)
cv2.waitKey()
cv2.destroyAllWindows()